# Rutschendes Masse-Feder-Masse System

Vor einer Wand ruhen zwei gleiche Körper mit jeweils der Masse m auf einem glatten, d.h. reibungsfreien Boden. Zwischen den Körpern befindet sich eine Feder mit Federkonstante $c$, die durch einen Faden um die Strecke $\Delta s$ zusammengedrückt wird. Zur Zeit $t=0$ wird der Faden durchgeschnitten.

```{image} images/mass-spring-mass.png
:alt: Mass-Spring-Mass Description
:width: 800px
:align: center
```

## Aufgabe 1: Physikalische Intuition

- Zu welcher Zeit $t_A$ löst sich der Körper 1 von der Wand und wie groß ist dabei die Schwerpunktsgeschwindigkeit $v_S$?
- Wie groß ist vermutlich die maximale Federdehnung $s_\text{max}$ der anschließenden Schwingung?

## Aufgabe 2: Berechnung der Bewegung

- Die Zeit $t_A$ durch Analyse einer Schwingungsbewegung allgemein gelöst werden. Danach nutzen Sie den Energieerhaltungssatz.
- Teil (b) aus Aufgabe 1: Wie groß ist der Schwerpunktsanteil, wie groß der Relativanteil an der Bewegungsenergie?

## Aufgabe 3: Lösung als ODE

Stellen Sie nun die ODE des Systems auf. Nutzen Sie zur Animation Ihrer Ergebnisse die unten stehenden Skripte.

**MATLAB Animation**

Inklusive GIF-Export zum Vergleich verschiedener Szenarien.

In [ ]:
function plotboxes(t,q,q0)
    filename = 'boxes_animated.gif';
    gifeverynth = 5;
    ploteverynth = 1;

    h = figure(1);
    boxdim = 1;

    ax = axes(h);
    set(ax,'xlim',[0 (max(q(:,2)+3*boxdim))])
    line(xlim, [0 0],'Color','black')
    axis equal

    box1 = [0 0; boxdim 0; boxdim boxdim; 0 boxdim] ;
    box2 = [0 0; boxdim 0; boxdim boxdim; 0 boxdim] + [(q0(2)+boxdim) 0];

    for i = 1:length(t)
        if mod(i-1,ploteverynth)==0
            if i > 1
                delete(p1);
                delete(p2);
                delete(spring);
            end
            P1 = box1+[q(i,1) 0];
            P2 = box2+[q(i,2) 0];
            if i == 1
                string = line([P1(3,1) P2(4,1)], [P1(3,2) P2(4,2)],'Color','red','Linestyle',':','Linewidth',2);
            end
            spring = line([P1(3,1) P2(4,1)], [P1(3,2) P2(4,2)]/2,'Color','black','Linestyle','-','Linewidth',2);
            p1 = patch(P1(:,1),P1(:,2),'r');
            p2 = patch(P2(:,1),P2(:,2),'r');
            title(['t = ',num2str(t(i))])
            drawnow
        end
        if q(i,3)<1e-3 && q(i,4)<1e-3 && q(i,1)>0
            title(['t = ',num2str(t(i)),', stopped plot due no further movement'])
            frame = getframe(h); 
            im = frame2im(frame); 
            [imind,cm] = rgb2ind(im,256);
            imwrite(imind,cm,filename,'gif','DelayTime',2,'WriteMode','append');
            break
        end
        if mod(i-1,gifeverynth)==0
            % Capture the plot as an image 
            frame = getframe(h); 
            im = frame2im(frame); 
            [imind,cm] = rgb2ind(im,256); 
            % Write to the GIF File 
            if i == 1
                imwrite(imind,cm,filename,'gif','DelayTime',2,'Loopcount',inf)
            else
                imwrite(imind,cm,filename,'gif','DelayTime',0.1,'WriteMode','append');
            end
        end
        if i == 1
            pause(1)
            delete(string);
        end
    end

    figure(2)
    plot(t,q(:,1),t,q(:,2))
    title('movements of boxes 1 and 2')
    legend('box 1','box 2')
    xlabel('time')
    ylabel('x')

end

**OCTAVE Animation**

```{admonition} Hinweis
Sollte das Schreiben der GIF-Datei sehr lange dauern, setzen Sie im Code `gifeverynth = 0` oder sehr hoch.
```

In [ ]:
function plotboxes_OCTAVE(t,q,q0)
    %graphics_toolkit (gnuplot)
    filename = 'boxes_animated.gif';
    gifeverynth = 20;
    ploteverynth = 5;

    h = figure(1);
    boxdim = 1;

    ax = axes(h);
    set(ax,'xlim',[0 (max(q(:,2)+3*boxdim))])
    axis equal

    box1 = [0 0; boxdim 0; boxdim boxdim; 0 boxdim] ;
    box2 = [0 0; boxdim 0; boxdim boxdim; 0 boxdim] + [(q0(2)+boxdim) 0];

    for i = 1:length(t)
        if mod(i-1,ploteverynth)==0
            if i > 1
                delete(p1);
                delete(p2);
                delete(spring);
            end
            P1 = box1+[q(i,1) 0];
            P2 = box2+[q(i,2) 0];
            if i == 1
                string = line([P1(3,1) P2(4,1)], [P1(3,2) P2(4,2)],'Color','red','Linestyle',':','Linewidth',2);
            end
            spring = line([P1(3,1) P2(4,1)], [P1(3,2) P2(4,2)]/2,'Color','black','Linestyle','-','Linewidth',2);
            p1 = patch(P1(:,1),P1(:,2),'r');
            p2 = patch(P2(:,1),P2(:,2),'r');
            title(['t = ',num2str(t(i))])
            drawnow
        end
        if q(i,3)<1e-3 && q(i,4)<1e-3 && q(i,1)>0
            title(['t = ',num2str(t(i)),', stopped plot due no further movement'])
            break
        end
        if gifeverynth > 0 && mod(i-1,gifeverynth)==0
            % Capture the plot as an image 
            frame = getframe(h); 
            im = frame2im(frame); 
            [imind,cm] = rgb2ind(im); 
            % Write to the GIF File 
            if i == 1
                imwrite(imind,cm,filename,'gif', 'Loopcount',inf,'Quality',10)%ind,cm
            else
                if i == gifeverynth
                    imwrite(imind,cm,filename,'gif','DelayTime',1,'WriteMode','append','Quality',10);
                else
                    imwrite(imind,cm,filename,'gif','DelayTime',0.1,'WriteMode','append','Quality',10);
                end
            end
        end
        if i == 1
            pause(1)
            delete(string);
        else
            pause(0.01)
        end
    end

    figure(2)
    plot(t,q(:,1),t,q(:,2))
    title('movements of boxes 1 and 2')
    legend('box 1','box 2')
    xlabel('time')
    ylabel('x')

end

Ihr Ergebnis könnte beispielsweise so aussehen ($l_0 = 1 \,\text{m}$, $c = 50 \,\frac{\text{N}}{\text{m}}$, $x_2(t=0) = 0.5 \,\text{m}$, $m_1 = m_2 = 2 \,\text{kg}$):

```{image} images/boxes_animated_std.gif
:alt: Mass-Spring-Mass Animated Standard GIF
:width: 600px
:align: center
```

```{image} images/boxes_animated_std.png
:alt: Mass-Spring-Mass Animated Standard Plot
:width: 600px
:align: center
```

Bei unterschiedlichen Massen sieht das Verhalten etwas anders aus (hier: $m_1 = 4 \,\text{kg}, m_2 = 2 \,\text{kg}$):

```{image} images/boxes_animated_massdiff.gif
:alt: Mass-Spring-Mass Animated Different Masses GIF
:width: 600px
:align: center
```

```{image} images/boxes_animated_massdiff.png
:alt: Mass-Spring-Mass Animated Different Masses Plot
:width: 600px
:align: center
```

## Aufgabe 4

- Erweitern Sie nun Ihre ODE Funktion so, dass Sie auch einen Reibungskoeffizienten mit einbeziehen können.
- Bedenken Sie, wovon die Reibkraft abhängt, wie und wann sie wirkt.
- Setzen Sie sinnvolle Abbruchbedingungen, um Oszillationen mit sehr geringer Bewegung zu vermeiden.

Für $\mu = 0.05$ und $m_1 = m_2 = 2 \,\text{kg}$ schwingt das System so:

```{image} images/boxes_animated_friction.gif
:alt: Mass-Spring-Mass Friciton GIF
:width: 600px
:align: center
```

```{image} images/boxes_animated_friction.png
:alt: Mass-Spring-Mass Animatex Friciton Plot
:width: 600px
:align: center
```